In [1]:
# syft absolute
import syft as sy
from syft import ActionObject
from syft.client.syncing import compare_states
from syft.client.syncing import resolve

kj/filesystem-disk-unix.c++:1734: warning: PWD environment variable doesn't match current directory; pwd = /Users/eelco/dev/PySyft


In [2]:
node_low = sy.orchestra.launch(
    name="test_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)
node_high = sy.orchestra.launch(
    name="test_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8a1c04544655402190588aec30079bc3.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=8ffc27886dbe48809f9f19d86e8874f3 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8212e6797fde4c3fba4fc53ab555a886.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=dafe6f7bace643439abae13975356590 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

/Users/eelco/dev/PySyft/packages/syft/src/syft/types/syft_object.py:599: TypeHintWarning: Skipping type check against 'ServiceRole'; this looks like a string-form forward reference imported from another module
  check_type(value, var_annotation)


Logged into <test_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
client_low.register(
    email="newuser@openmined.org", name="John Doe", password="pw", password_verify="pw"
)
client_low_ds = node_low.login(email="newuser@openmined.org", password="pw")

Logged into <test_l: Low side Domain> as <newuser@openmined.org>


# create datasets

In [5]:
# third party
import numpy as np

In [6]:
mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client_high.upload_dataset(dataset_high)

  0%|                                                                                             | 0/1 [00:00<?, ?it/s]

Uploading: numpy-data


100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.37it/s]


ADDING STORE PERMISSION ON SET 7985d642d9e54cbfad44415b32a72027 8212e6797fde4c3fba4fc53ab555a886
syft_node_location=None syft_client_verify_key=None output={'id': <UID: 6af075e77fbc4ebebc258ffab3590ac1>, 'name': 'numpy-data', 'description': syft.service.dataset.dataset.MarkdownDescription, 'contributors': {syft.service.dataset.dataset.Contributor}, 'data_subjects': [], 'node_uid': <UID: 8212e6797fde4c3fba4fc53ab555a886>, 'action_id': <UID: 7985d642d9e54cbfad44415b32a72027>, 'data': array([15, 16, 17, 18, 19]), 'mock': array([10, 11, 12, 13, 14]), 'shape': (5,), 'mock_is_real': True, 'created_at': syft.types.datetime.DateTime, 'uploader': syft.service.dataset.dataset.Contributor} node=Domain: test_h - 8212e6797fde4c3fba4fc53ab555a886 - domain

Services:
ActionService, BlobStorageService, CodeHistoryService, DataSubjectMemberService, DataSubjectService, DatasetService, EnclaveService, JobService, LogService, MetadataService, MigrateStateService, NetworkService, NotificationService, Outpu

SyftSuccess: Dataset uploaded to 'test_h'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [7]:
mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
# private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    id=dataset_high.id,
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=ActionObject.empty(data_node_id=client_high.id),
            shape=mock_low.shape,
            mock_is_real=True,
        )
    ],
)

res = client_low.upload_dataset(dataset_low)

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 18.74it/s]

Uploading: numpy-data
ADDING STORE PERMISSION ON SET eb356a4e8e214848bedb86822a03ce40 8a1c04544655402190588aec30079bc3


syft_node_location=None syft_client_verify_key=None output={'id': <UID: d31b59d1394e49feac20aacb1695e47d>, 'name': 'numpy-data', 'description': syft.service.dataset.dataset.MarkdownDescription, 'contributors': {syft.service.dataset.dataset.Contributor}, 'data_subjects': [], 'node_uid': <UID: 8a1c04544655402190588aec30079bc3>, 'action_id': <UID: eb356a4e8e214848bedb86822a03ce40>, 'data': Pointer:
None, 'mock': array([0, 1, 2, 3, 4]), 'shape': (5,), 'mock_is_real': True, 'created_at': syft.types.datetime.DateTime, 'uploader': syft.service.dataset.dataset.Contributor} node=Domain: test_l - 8a1c04544655402190588aec30079bc3 - domain

Services:
ActionService, BlobStorageService, CodeHistoryService, DataSubjectMemberService, DataSubjectService, DatasetService, EnclaveService, JobService, LogService, MetadataService, MigrateStateService, NetworkService, NotificationService, OutputService, PolicyService, ProjectService, QueueService, RequestService, SettingsService, SyftImageRegistryService, Sy

In [8]:
dataset_storage_permissions = node_low.python_node.get_service(
    "datasetservice"
).stash.partition.storage_permissions
dataset_id = client_low.datasets[0].id

storage_permissions = dataset_storage_permissions[dataset_id]
assert storage_permissions == {node_low.python_node.id}

# Make Requests

In [9]:
data_low = client_low_ds.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    print("Computing mean...")
    return data.mean()


compute_mean(data=data_low.mock)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftInfo: Creating a node with n_consumers=2 (the default value)
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/78698220bb7b41799df061e06dd6d721.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=2 image_uid=466fb5ab122b4e798be1ec27a917d0e5 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Logged into <ephemeral_node_compute_mean_4541: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

ADDING STORE PERMISSION ON SET eb356a4e8e214848bedb86822a03ce40 78698220bb7b41799df061e06dd6d721
Approving request for domain ephemeral_node_compute_mean_4541
ADDING STORE PERMISSION ON SET d843d5375e0d46ceaffe741cf2f75d73 78698220bb7b41799df061e06dd6d721
Computing mean...
ADDING STORE PERMISSION ON SET e86fbd59d6c74cf28f284b435f4928a8 78698220bb7b41799df061e06dd6d721
SyftInfo: Landing the ephmeral node...


```python
Pointer
```
2.0

In [10]:
client_low_ds.code.request_code_execution(compute_mean)

SyftInfo: Node Landed!


syft.service.request.request.Request

## Sync to high side

In [11]:
low_state = client_low.get_sync_state()
high_state = client_high.get_sync_state()

In [12]:
assert (
    set(low_state.objects.keys())
    == set(low_state.permissions.keys())
    == set(low_state.storage_permissions.keys())
)

In [13]:
low_state

syft.service.sync.sync_state.SyncState

In [14]:
diff_state = compare_states(low_state, high_state)

In [15]:
print(diff_state.diffs[0].to_dict())

{'id': <UID: 06911b2f4c1b405d90a872462ede650a>, 'low_obj': {'id': <UID: 8186658a7012479d89b25822a8000888>, 'from_mock_sync': False}, 'high_obj': None, 'low_node_uid': <UID: 8a1c04544655402190588aec30079bc3>, 'high_node_uid': <UID: 8212e6797fde4c3fba4fc53ab555a886>, 'low_permissions': ['bfae60839f689c856037b1da0ae406b21b497046233a3592310399f103623f6c_WRITE', 'bfae60839f689c856037b1da0ae406b21b497046233a3592310399f103623f6c_OWNER', 'bfae60839f689c856037b1da0ae406b21b497046233a3592310399f103623f6c_READ', 'bfae60839f689c856037b1da0ae406b21b497046233a3592310399f103623f6c_EXECUTE'], 'high_permissions': [], 'low_storage_permissions': {<UID: 8a1c04544655402190588aec30079bc3>}, 'high_storage_permissions': set(), 'obj_type': <class 'syft.service.code.user_code.UserCodeStatusCollection'>, 'diff_list': []}


In [16]:
resolved_state_low, resolved_state_high = resolve(diff_state, decision="low")

LOW SIDE STATE:

―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #d2b3d9b028814f2a9c3b82b383d53782:
service_func_name: compute_mean
input_owners: [
‎ ‎ ‎ ‎ test_l
]
code_status: [
‎ ‎ ‎ ‎ Node: test_l, Status: pending
]
worker_pool_name: default-pool

    ―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODESTATUSCOLLECTION #8186658a7012479d89b25822a8000888:
    approved: False
    status_dict: {
    ‎ ‎ ‎ ‎ node_id=<UID: 8a1c04544655402190588aec30079bc3>
        verify_key=1c0590b894d19530970da6bf09bb302afa3e473a4cccdacbace0e20223ee3367
        node_name='test_l': (<UserCodeStatus.PENDING: 'pending'>, '')
    }



HIGH SIDE STATE:

No high side changes.

Decision: Syncing 2 objects from low side


LOW SIDE STATE:

―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

REQUEST #76b502afc9994d3b9d5dcff0b0f725d8:
request_ti

In [17]:
print("Resolved state low side")
print(resolved_state_low)
print()
print("Resolved state high side")
print(resolved_state_high)

Resolved state low side
ResolvedSyncState(
  create_objs=[],
  update_objs=[],
  delete_objs=[]
  new_permissions=[]
)

Resolved state high side
ResolvedSyncState(
  create_objs=[{NodeIdentity <name=test_l, id=8a1c0454, 🔑=1c0590b8>: (<UserCodeStatus.PENDING: 'pending'>, '')}, syft.service.code.user_code.UserCode, syft.service.request.request.Request],
  update_objs=[],
  delete_objs=[]
  new_permissions=[]
)


In [18]:
resolved_state_high.new_permissions

[]

In [19]:
client_low.apply_state(resolved_state_low)

/Users/eelco/dev/PySyft/packages/syft/src/syft/types/syft_object.py:599: TypeHintWarning: Skipping type check against 'ServiceRole'; this looks like a string-form forward reference imported from another module
  check_type(value, var_annotation)


SyftSuccess: Synced 0 items

In [20]:
client_high.apply_state(resolved_state_high)

SyftSuccess: Synced 3 items

# Run code high and sync back result

In [21]:
data_high = client_high.datasets[0].assets[0]

In [22]:
job_high = client_high.code.compute_mean(data=data_high, blocking=False)
display(job_high)

ADDING STORE PERMISSION ON SET a08f545d3c3e427b8a6e9550e707012f 8212e6797fde4c3fba4fc53ab555a886


```python
class Job:
    id: UID = 5c6b5683fea540d9ab54da2ea192e032
    status: created
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

In [23]:
# wait for the result
job_high.wait().get()

ADDING STORE PERMISSION ON SET 7985d642d9e54cbfad44415b32a72027 8212e6797fde4c3fba4fc53ab555a886


07/03/24 15:32:42 FUNCTION LOG (5c6b5683fea540d9ab54da2ea192e032): Computing mean...


ADDING STORE PERMISSION ON SET a08f545d3c3e427b8a6e9550e707012f 8212e6797fde4c3fba4fc53ab555a886


17.0

In [24]:
job_info = job_high.info(public_metadata=True, result=True)

request = client_high.requests[0]
result_obj = job_high.result

In [25]:
# syft absolute
from syft import SyftError
from syft import SyftSuccess

# Accepting the result directly gives an error
accept_res = request.accept_by_depositing_result(result_obj)
assert isinstance(accept_res, SyftError)

In [26]:
accept_res = request.accept_by_depositing_result(job_info)

assert isinstance(accept_res, SyftSuccess)
accept_res

Approving request for domain test_h
returning existing job
setting permission
None
None
Approving request for domain test_h
ADDING PERMISSION [READ: a08f545d3c3e427b8a6e9550e707012f as bfae60839f689c856037b1da0ae406b21b497046233a3592310399f103623f6c] a08f545d3c3e427b8a6e9550e707012f
ADDING STORE PERMISSION ON SET 663c49413f224c20b0d41201571c57e8 8212e6797fde4c3fba4fc53ab555a886
Job(5c6b5683fea540d9ab54da2ea192e032) Setting new result a08f545d3c3e427b8a6e9550e707012f -> a08f545d3c3e427b8a6e9550e707012f


SyftSuccess: Request submitted for updating result.

In [27]:
accept_res

SyftSuccess: Request submitted for updating result.

In [28]:
request.code.output_history[0]

```python
class ExecutionOutput:
  id: str = dac663703db542c4b0e3bcb99a092dde
  created_at: str = 2024-03-07 13:32:27
  user_code_id: str = d2b3d9b028814f2a9c3b82b383d53782
  job_id: str = 5c6b5683fea540d9ab54da2ea192e032
  output_ids: str = [<LineageID: a08f545d3c3e427b8a6e9550e707012f - 1074451811235599667>]

```

In [29]:
# accept_res = request.accept_by_depositing_result(job_info.result.get())

# assert isinstance(accept_res, SyftSuccess)
# accept_res

In [30]:
# Need to refresh Job because we overwrite result
job_high = client_high.jobs[0]

action_store_high = node_high.python_node.get_service("actionservice").store
blob_store_high = node_high.python_node.get_service(
    "blobstorageservice"
).stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_high.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_high.permissions[job_high.result.syft_blob_storage_entry_id]
)

## Sync back to low

In [31]:
client_high.jobs[0]

```python
class Job:
    id: UID = 5c6b5683fea540d9ab54da2ea192e032
    status: completed
    has_parent: False
    result: 17.0
    logs:

0 Computing mean...
JOB COMPLETED
    
```

In [32]:
low_state = client_low.get_sync_state()
high_state = client_high.get_sync_state()

high_state

syft.service.sync.sync_state.SyncState

In [33]:
high_state.storage_permissions

{<UID: 76b502afc9994d3b9d5dcff0b0f725d8>: {<UID: 8212e6797fde4c3fba4fc53ab555a886>},
 <UID: d2b3d9b028814f2a9c3b82b383d53782>: {<UID: 8212e6797fde4c3fba4fc53ab555a886>},
 <UID: 5c6b5683fea540d9ab54da2ea192e032>: {<UID: 8212e6797fde4c3fba4fc53ab555a886>},
 <UID: 5a25803cac37443884e29985354d7244>: {<UID: 8212e6797fde4c3fba4fc53ab555a886>},
 <UID: dac663703db542c4b0e3bcb99a092dde>: {<UID: 8212e6797fde4c3fba4fc53ab555a886>},
 <UID: 8186658a7012479d89b25822a8000888>: {<UID: 8212e6797fde4c3fba4fc53ab555a886>},
 <UID: a08f545d3c3e427b8a6e9550e707012f>: {<UID: 8212e6797fde4c3fba4fc53ab555a886>}}

In [34]:
diff_state_2 = compare_states(low_state, high_state)

diff_state_2

syft.service.sync.diff_state.NodeDiff

In [35]:
resolved_state_low, resolved_state_high = resolve(
    diff_state_2, decision="high", share_private_objects=False
)

LOW SIDE STATE:

―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #d2b3d9b028814f2a9c3b82b383d53782:
service_func_name: compute_mean
input_owners: [
‎ ‎ ‎ ‎ test_l
]
code_status: [
‎ ‎ ‎ ‎ Node: test_l, Status: pending
]

    ―――― DIFF ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODESTATUSCOLLECTION #8186658a7012479d89b25822a8000888:
    status_dict: (<UserCodeStatus.PENDING: 'pending'>, '')



HIGH SIDE STATE:

―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #d2b3d9b028814f2a9c3b82b383d53782:
service_func_name: compute_mean
input_owners: [
‎ ‎ ‎ ‎ test_l
]
code_status: [
‎ ‎ ‎ ‎ Node: test_h, Status: approved
]

    ―――― DIFF ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODESTATUSCOLLECTION #8186658a7012479d89b25822a8000888:
    status_dict: (<UserCodeStatus.AP

 a08




Setting permissions for NumpyScalarObject #a08f545d3c3e427b8a6e9550e707012f to share with bfae60839f689c856037b1da0ae406b21b497046233a3592310399f103623f6c,
this will become effective when you call client.apply_state(<resolved_state>))

You currently have the following private objects:

SyftLog #5a25803cac37443884e29985354d7244

Do you want to share some of these private objects? If so type the first 3 characters of the id e.g. 'abc'.
If you dont want to share any more private objects, type "no"



 5a2




Setting permissions for SyftLog #5a25803cac37443884e29985354d7244 to share with bfae60839f689c856037b1da0ae406b21b497046233a3592310399f103623f6c,
this will become effective when you call client.apply_state(<resolved_state>))

Decision: Syncing 5 objects from high side




In [36]:
resolved_state_low.new_storage_permissions

[StoragePermission: 8186658a7012479d89b25822a8000888 on 8a1c04544655402190588aec30079bc3,
 StoragePermission: 76b502afc9994d3b9d5dcff0b0f725d8 on 8a1c04544655402190588aec30079bc3,
 StoragePermission: dac663703db542c4b0e3bcb99a092dde on 8a1c04544655402190588aec30079bc3,
 StoragePermission: a08f545d3c3e427b8a6e9550e707012f on 8a1c04544655402190588aec30079bc3,
 StoragePermission: 5c6b5683fea540d9ab54da2ea192e032 on 8a1c04544655402190588aec30079bc3,
 StoragePermission: 5a25803cac37443884e29985354d7244 on 8a1c04544655402190588aec30079bc3]

In [37]:
print(resolved_state_low)
print()
print(resolved_state_high)

ResolvedSyncState(
  create_objs=[syft.service.output.output_service.ExecutionOutput, Pointer:
17.0, syft.service.job.job_stash.Job, syft.service.log.log.SyftLog],
  update_objs=[{NodeIdentity <name=test_h, id=8212e679, 🔑=d311a667>: (<UserCodeStatus.APPROVED: 'approved'>, '')}, syft.service.request.request.Request],
  delete_objs=[]
  new_permissions=[[READ: a08f545d3c3e427b8a6e9550e707012f as bfae60839f689c856037b1da0ae406b21b497046233a3592310399f103623f6c], [READ: 5c6b5683fea540d9ab54da2ea192e032 as bfae60839f689c856037b1da0ae406b21b497046233a3592310399f103623f6c], [READ: 5a25803cac37443884e29985354d7244 as bfae60839f689c856037b1da0ae406b21b497046233a3592310399f103623f6c]]
)

ResolvedSyncState(
  create_objs=[],
  update_objs=[],
  delete_objs=[]
  new_permissions=[]
)


In [38]:
client_low.apply_state(resolved_state_low)

ADDING STORE PERMISSION ON SET a08f545d3c3e427b8a6e9550e707012f 8a1c04544655402190588aec30079bc3


/Users/eelco/dev/PySyft/packages/syft/src/syft/types/syft_object.py:599: TypeHintWarning: Skipping type check against 'ServiceRole'; this looks like a string-form forward reference imported from another module
  check_type(value, var_annotation)


SyftSuccess: Synced 6 items

In [39]:
client_high.apply_state(resolved_state_high)

SyftSuccess: Synced 0 items

In [40]:
action_store_low = node_low.python_node.get_service("actionservice").store
blob_store_low = node_low.python_node.get_service("blobstorageservice").stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_low.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_low.permissions[job_high.result.syft_blob_storage_entry_id]
)

In [42]:
result_storage_permissions = node_low.python_node.get_service(
    "actionservice"
).store.storage_permissions[job_high.result.id.id]
assert len(result_storage_permissions) == 1

log_storage_permissions = node_low.python_node.get_service(
    "logservice"
).stash.partition.storage_permissions[job_high.log_id]
assert len(log_storage_permissions) == 1

# Run code low

## Run

In [45]:
# syft absolute
from syft import UID

uid = UID.from_string("4f6a397382204206af8dcac13a9d508c")

In [46]:
client_low_ds._fetch_api(client_low.credentials)

/Users/eelco/dev/PySyft/packages/syft/src/syft/types/syft_object.py:599: TypeHintWarning: Skipping type check against 'ServiceRole'; this looks like a string-form forward reference imported from another module
  check_type(value, var_annotation)


In [47]:
res_low = client_low_ds.code.compute_mean(data=data_low)

res_low

```python
Pointer
```
17.0

In [48]:
res_low.id == job_high.result.id

True

In [49]:
code = client_low_ds.code[0]

assert res_low.get() == private_high.mean()
assert (
    res_low.id.id
    == job_high.result.id.id
    == code.output_history[-1].output_ids[0].id.id
)
assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

In [50]:
res_low = client_low_ds.code.compute_mean(data=data_low)

res_low

```python
Pointer
```
17.0

In [51]:
code = client_low_ds.code[0]

assert res_low.get() == private_high.mean()
assert (
    res_low.id.id
    == job_high.result.id.id
    == code.output_history[-1].output_ids[0].id.id
)
assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

In [52]:
private_high.mean()

17.0

In [53]:
job_low = client_low_ds.code.compute_mean(data=data_low, blocking=False)
job_low

SyftWarning: There are existing jobs for this user code, returning the latest one

```python
class Job:
    id: UID = 5c6b5683fea540d9ab54da2ea192e032
    status: completed
    has_parent: False
    result: ActionDataEmpty <<class 'numpy.number'>>
    logs:

0 Log 5a25803cac37443884e29985354d7244 not available
JOB COMPLETED
    
```

In [54]:
job_low.wait().get()

17.0

In [55]:
job_low.logs()

Log 5a25803cac37443884e29985354d7244 not available


In [56]:
job_high.result.id

<LineageID: a08f545d3c3e427b8a6e9550e707012f - 1074451811235599667>

In [57]:
job_high.result.as_empty().id

<LineageID: a08f545d3c3e427b8a6e9550e707012f - 1074451811235599667>

In [58]:
assert job_low.id == job_high.id
assert job_low.result.id == job_high.result.id
assert (
    job_low.result.syft_blob_storage_entry_id
    == job_high.result.syft_blob_storage_entry_id
)

In [59]:
job_high.result.id

<LineageID: a08f545d3c3e427b8a6e9550e707012f - 1074451811235599667>

In [60]:
job_low.result.id

<LineageID: a08f545d3c3e427b8a6e9550e707012f - 1074451811235599667>

In [61]:
job_high

```python
class Job:
    id: UID = 5c6b5683fea540d9ab54da2ea192e032
    status: completed
    has_parent: False
    result: 17.0
    logs:

0 Computing mean...
JOB COMPLETED
    
```

Timeout elapsed after 30 seconds while trying to acquiring lock.
FAILED TO LOCK
Timeout elapsed after 30 seconds while trying to acquiring lock.
FAILED TO LOCK
Timeout elapsed after 30 seconds while trying to acquiring lock.
FAILED TO LOCK
Timeout elapsed after 30 seconds while trying to acquiring lock.
FAILED TO LOCK
Timeout elapsed after 30 seconds while trying to acquiring lock.
FAILED TO LOCK
Timeout elapsed after 30 seconds while trying to acquiring lock.
FAILED TO LOCK
Timeout elapsed after 30 seconds while trying to acquiring lock.
FAILED TO LOCK
Timeout elapsed after 30 seconds while trying to acquiring lock.
FAILED TO LOCK
Timeout elapsed after 30 seconds while trying to acquiring lock.
FAILED TO LOCK
Timeout elapsed after 30 seconds while trying to acquiring lock.
FAILED TO LOCK
Timeout elapsed after 30 seconds while trying to acquiring lock.
FAILED TO LOCK
Timeout elapsed after 30 seconds while trying to acquiring lock.
FAILED TO LOCK
Timeout elapsed after 30 seconds while t